# Image Deconvolution
- ver.0.0.1
- 2023 Apr 21
- H. Yoneda
- using 1s GRB simulation data from source_injector.ipynb

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from mhealpy import HealpixMap
from cosipy.image_deconvolution import dataIO
from cosipy.image_deconvolution import ImageDeconvolution

In [ ]:
from cosipy import test_data

In [ ]:
event_filepath = test_data.path / "data_pl.h5"
bkg_filepath = test_data.path / "bkg_pl.h5"
response_filepath = test_data.path / "FlatContinuumIsotropic.LowRes.binnedimaging.imagingresponse.area.nside8.cosipy.h5"


duration = 1 * u.s

## load files

- Need the dataIO library for the event and background files

In [ ]:
dataloader = dataIO(event_filepath = event_filepath, 
                    bkg_filepath = bkg_filepath, 
                    response_filepath = response_filepath, 
                    duration = duration)

In [ ]:
dataloader.response.axes["Ei"].edges

In [ ]:
dataloader.response.axes.labels

In [ ]:
dataloader.response.axes['Phi'].widths

## load data

In [ ]:
image_deconvolution = ImageDeconvolution()

In [ ]:
image_deconvolution.set_data(dataloader)

## load parameterfile

In [ ]:
parameter_filepath = test_data.path / "imagedeconvolution_parfile_itr10.yml"

image_deconvolution.read_parameterfile(parameter_filepath)

## sparse matrix?

In [ ]:
image_deconvolution.use_sparse = False

## initialization
- produce an initial flux map etc.

In [ ]:
image_deconvolution.initialize()

## run deconvolution

In [ ]:
%%time

all_result = image_deconvolution.run_deconvolution()

## analyze the result


In [ ]:
print(all_result)

### likelihood

In [ ]:
x, y = [], []

for _ in all_result:
    x.append(_['iteration'])
    y.append(_['loglikelihood'])
    
plt.plot(x, y)
plt.xlabel("iteration")
plt.ylabel("loglikelihood")

### alpha

In [ ]:
x, y = [], []

for _ in all_result:
    x.append(_['iteration'])
    y.append(_['alpha'])
    
plt.plot(x, y)
plt.xlabel("iteration")
plt.ylabel("alpha")

### image

In [ ]:
for _ in all_result:
    iteration = _['iteration']
    image = _['model_map']
        
    for energy_band in range(5):
        map_healpxmap = HealpixMap(data = image[:,energy_band], unit = image.unit)

        map_healpxmap.plot()
        plt.title(label = "iteration = {}, energyband = {}".format(iteration, energy_band))
        plt.show()

### delta_image

In [ ]:
for _ in all_result:
    iteration = _['iteration']
    image = _['delta_map']
        
    for energy_band in range(5):
        map_healpxmap = HealpixMap(data = image[:,energy_band], unit = image.unit)

        map_healpxmap.plot()
        plt.title(label = "iteration = {}, energyband = {}".format(iteration, energy_band))
        plt.show()

### flux

In [ ]:
iteration = []
integrated_flux = []
integrated_flux_each_band = [[],[],[],[],[]]

for _ in all_result:
    iteration.append(_['iteration'])
    image = _['model_map']

    integrated_flux.append( np.sum(image) / image.unit)

    for energy_band in range(5):
        integrated_flux_each_band[energy_band].append( np.sum(image[:,energy_band]) / image.unit)
        
plt.plot(iteration, integrated_flux, label = 'total')
plt.xlabel("iteration")
plt.ylabel("integrated flux")
plt.yscale("log")

for energy_band in range(5):
    plt.plot(iteration, integrated_flux_each_band[energy_band], label = "energyband = {}".format(energy_band))
    
plt.legend()

### flux delta

In [ ]:
iteration = []
delta_integrated_flux = []
delta_integrated_flux_each_band = [[],[],[],[],[]]

for _ in all_result:
    iteration.append(_['iteration'])
    delta_image = _['delta_map']

    delta_integrated_flux.append( np.sum(delta_image) / delta_image.unit)

    for energy_band in range(5):
        delta_integrated_flux_each_band[energy_band].append( np.sum(delta_image[:,energy_band]) / delta_image.unit)
        
plt.plot(iteration, delta_integrated_flux, label = 'total')
plt.xlabel("iteration")
plt.ylabel("delta integrated flux")
plt.ylim(-0.08, 0.08)

for energy_band in range(5):
    plt.plot(iteration, delta_integrated_flux_each_band[energy_band], label = "energyband = {}".format(energy_band))
    
plt.legend()

## comparision with the truth value

In [ ]:
from gammapy.modeling.models import PowerLawSpectralModel

spectrum = PowerLawSpectralModel(index = 1.7,
                                 amplitude = 1e-1 / u.cm / u.cm / u.s / u.keV,
                                 reference = 1 * u.keV)

integrated_flux_truth = spectrum.integral(100 * u.keV, 5000.0 * u.keV)
pixel_area = 4 * 3.1415 * u.sr / all_result[0]['model_map'].axes['NuLambda'].npix

print("truth:", integrated_flux_truth)

print("deconvolution:", integrated_flux[-1] * pixel_area * u.cm**-2 * u.s**-1 * u.sr**-1)

In [ ]:
integrated_flux_each_band_truth = []

for energy_band in range(5):
    emin, emax = dataloader.response.axes["Ei"].bounds[energy_band]

    integrated_flux_each_band_truth.append(spectrum.integral(emin, emax))
    pixel_area = 4 * 3.1415 * u.sr / all_result[0]['model_map'].axes['NuLambda'].npix
    
    print(emin, emax)
    print("    truth:", integrated_flux_each_band_truth[energy_band])
    print("    deconvolution:", integrated_flux_each_band[energy_band][-1] * pixel_area * u.cm**-2 * u.s**-1 * u.sr**-1)

In [ ]:
iteration = []
integrated_flux = []
integrated_flux_each_band = [[],[],[],[],[]]

for _ in all_result:
    iteration.append(_['iteration'])
    image = _['model_map']

    integrated_flux.append( np.sum(image) / image.unit)

    for energy_band in range(5):
        integrated_flux_each_band[energy_band].append( np.sum(image[:,energy_band]) / image.unit)
        
plt.plot(iteration, integrated_flux * pixel_area, label = 'total', color = 'black')
plt.plot(iteration, np.full(len(iteration), integrated_flux_truth), color = 'black', linestyle = "--")
plt.xlabel("iteration")
plt.ylabel("integrated flux")
plt.yscale("log")


colors = ['b', 'g', 'r', 'c', 'm']
for energy_band in range(5):
    plt.plot(iteration, integrated_flux_each_band[energy_band] * pixel_area, label = "energyband = {}".format(energy_band), color = colors[energy_band])
    plt.plot(iteration, np.full(len(iteration), integrated_flux_each_band_truth[energy_band]), color = colors[energy_band], linestyle = "--")
    
plt.legend()